In [1]:
!pip install langchain_community

In [3]:
import torch
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          pipeline, TextStreamer)
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

MODEL_NAME = "microsoft/phi-3-mini-4k-instruct"      # or "HuggingFaceH4/zephyr-7b-beta"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto"
)

# --- Text-generation pipeline --------------------------------
gen_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=120,       # only generate up to 120 fresh tokens
    do_sample=True,
    temperature=0.9,
    top_p=0.95,
    repetition_penalty=1.15,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    streamer=TextStreamer(tokenizer, skip_prompt=True)
)

llm = HuggingFacePipeline(pipeline=gen_pipe)

# --- ChatML-style prompt (Phi-3/Zephyr understand this natively) ----
AD_PROMPT = PromptTemplate(
    input_variables=["bike_specs", "discount_options", "theme"],
    template=(
        "<|system|>\n"
        "You are an award-winning copywriter who writes vibrant, eco-friendly ads.\n<|end|>\n"
        "<|user|>\n"
        "Write a punchy bike-rental advert (≤ 70 words).\n"
        "Theme: {theme}\n"
        "Bike features: {bike_specs}\n"
        "Promos: {discount_options}\n"
        "End with a strong call-to-action.\n<|end|>\n"
        "<|assistant|>"
    )
)

ad_chain = LLMChain(llm=llm, prompt=AD_PROMPT)

def generate_bike_rental_ad(bike_specs: str, discount_options: str, theme: str) -> str:
    """Return a short, engaging bike-rental advertisement."""
    return ad_chain.run(
        bike_specs=bike_specs,
        discount_options=discount_options,
        theme=theme
    ).strip()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu
<ipython-input-3-fd1a2a05317e>:31: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=gen_pipe)
<ipython-input-3-fd1a2a05317e>:49: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  ad_chain = LLMChain(llm=llm, prompt=AD_PROMPT)


In [4]:
# Example usage
bike_specs = input("Enter bike specifications (e.g., Electric, Lightweight, GPS Enabled): ")
discount_options = input("Enter discount options (e.g., 20% off for first-time users): ")
theme = input("Enter ad theme (e.g., Adventure in the City, Family Fun, Eco-Friendly Travel): ")
ad = generate_bike_rental_ad(bike_specs, discount_options, theme)
print("Generated ad:")
print(ad)

Enter bike specifications (e.g., Electric, Lightweight, GPS Enabled): Electric
Enter discount options (e.g., 20% off for first-time users): 50% off
Enter ad theme (e.g., Adventure in the City, Family Fun, Eco-Friendly Travel): Eco-friendly travel


<ipython-input-3-fd1a2a05317e>:53: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return ad_chain.run(


Pedaling for planet'tastical! Grab your electric ride at GreenWheelz—whisk through the city on two wheels as if you were born to it! With our "Pedals & Planet" promotion slashing prices by half today only, glide into green living now! Leave gas guzzlers behind; join us and cycle toward sustainability – book online NOW before we roll out of stock like autumn leaves in cyclone speed!!! Ride smart... Live greener… Get yours this hour! #ElectricRidesForChange��
Generated ad:
<|system|>
You are an award-winning copywriter who writes vibrant, eco-friendly ads.
<|end|>
<|user|>
Write a punchy bike-rental advert (≤ 70 words).
Theme: Eco-friendly travel
Bike features: Electric
Promos: 50% off
End with a strong call-to-action.
<|end|>
<|assistant|> Pedaling for planet'tastical! Grab your electric ride at GreenWheelz—whisk through the city on two wheels as if you were born to it! With our "Pedals & Planet" promotion slashing prices by half today only, glide into green living now! Leave gas guzzle